1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
%pip install --upgrade --quiet  docx2txt
%pip install --upgrade --quiet  langchain-community

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

document_list

In [ ]:
%pip install langchain-chroma

In [5]:
from langchain_chroma import Chroma

database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="tax", persist_directory="./chroma")


In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가?'
retrieved_docs = database.similarity_search(query, k=3)
retrieved_docs

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini")

# prompt = f"""[Identity]

# - 당신은 최고의 한국 소득세 전문가 입니다.
# - [Context]를 참고해서 사용자의 질문에 답변해주세요.

# [Context]
# {retrieved_docs}

# Question: {query}
# """

# ai_message = llm.invoke(prompt)
# ai_message.content

In [ ]:
%pip install --upgrade --quiet langchain-community langchain langchain-openai faiss-cpu

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma

from dotenv import load_dotenv

load_dotenv()

# 모델 설정
llm = ChatOpenAI(model_name="gpt-4o-mini")

# Knowledge Base 구성을 위한 데이터 생성
# - RecursiveCharacterTextSplitter를 활용한 데이터 chunking
#   - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
#   - 비용 감소와 답변 생성시간 감소의 효과
#   - LangChain에서 다양한 TextSplitter들을 제공
# - chunk_size 는 split 된 chunk의 최대 크기
# - chunk_overlap은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

# 데이터를 처음 저장할 때 
# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="tax", persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때 
database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

# 답변 생성을 위한 Retrieval
query = '연봉 5천만원인 직장인의 소득세는 얼마인가?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

# Augmentation을 위한 Prompt 활용
# - Retrieval된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용
system_prompt = f"""[Identity]

- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
"{retrieved_docs}"

Question: {query}
"""
prompt = hub.pull("rlm/rag-prompt")

# 답변 생성
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain.invoke({"query": query})
ai_message